In [1]:
import os
from glob import glob
import shutil
from tqdm import tqdm
# import dicom2nifti
import numpy as np
# import nibabel as nib
from monai.transforms import(
    Compose,
    # AddChannel,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism

In [2]:
def create_groups(in_dir, out_dir, Number_slices):
    '''
    This function is to get the last part of the path so that we can use it to name the folder.
    `in_dir`: the path to your folders that contain dicom files
    `out_dir`: the path where you want to put the converted nifti files
    `Number_slices`: here you put the number of slices that you need for your project and it will 
    create groups with this number.
    '''

    for patient in glob(in_dir + '/*'):
        patient_name = os.path.basename(os.path.normpath(patient))

        # Here we need to calculate the number of folders which mean into how many groups we will divide the number of slices
        number_folders = int(len(glob(patient + '/*')) / Number_slices)

        for i in range(number_folders):
            output_path = os.path.join(out_dir, patient_name + '_' + str(i))
            os.mkdir(output_path)

            # Move the slices into a specific folder so that you will save memory in your desk
            for i, file in enumerate(glob(patient + '/*')):
                if i == Number_slices + 1:
                    break
                
                shutil.move(file, output_path)

In [4]:
create_groups('/Users/tylerklimas/Desktop/LiverSegmentation/LiverSegmentationData', '/Users/tylerklimas/Desktop/LiverSegmentation/LiverSegmentationData/groups', 3)

In [40]:
def prepare(in_dir, pixdim=(1.5, 1.5, 1.0), a_min=-200, a_max=200, spatial_size=[128,128,64], cache=False):

    """
    This function is for preprocessing, it contains only the basic transforms, but you can add more operations that you 
    find in the Monai documentation.
    https://monai.io/docs.html
    """
    files = sorted(os.listdir(in_dir))
    print(len(files))
#     print("Files in Directory:", files)
    set_determinism(seed=0)

    path_train_volumes = sorted(glob(os.path.join(in_dir, "groups", "*.nii")))
#     print(path_train_volumes)
    
    path_train_segmentation = sorted(glob(os.path.join(in_dir, "groups", "*.nii")))

    path_test_volumes = sorted(glob(os.path.join(in_dir, "groups", "*.nii.gz")))
    path_test_segmentation = sorted(glob(os.path.join(in_dir, "groups", "*.nii.gz")))

    train_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(files, files)]
    test_files = [{"vol": image_name, "seg": label_name} for image_name, label_name in zip(path_test_volumes, path_test_segmentation)]
    
#     print(train_files)
    train_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            # AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max, b_min=0.0, b_max=1.0, clip=True), 
            CropForegroundd(keys=["vol", "seg"], source_key="vol"),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),   
            ToTensord(keys=["vol", "seg"]),

        ]
    )


    test_transforms = Compose(
        [
            LoadImaged(keys=["vol", "seg"]),
            # AddChanneld(keys=["vol", "seg"]),
            Spacingd(keys=["vol", "seg"], pixdim=pixdim, mode=("bilinear", "nearest")),
            Orientationd(keys=["vol", "seg"], axcodes="RAS"),
            ScaleIntensityRanged(keys=["vol"], a_min=a_min, a_max=a_max,b_min=0.0, b_max=1.0, clip=True), 
            CropForegroundd(keys=['vol', 'seg'], source_key='vol', allow_smaller=False),
            Resized(keys=["vol", "seg"], spatial_size=spatial_size),   
            ToTensord(keys=["vol", "seg"]),

            
        ]
    )

    if cache:
        train_ds = CacheDataset(data=train_files, transform=train_transforms,cache_rate=1.0)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = CacheDataset(data=test_files, transform=test_transforms, cache_rate=1.0)
        test_loader = DataLoader(test_ds, batch_size=1)

        return train_loader, test_loader

    else:
        train_ds = Dataset(data=train_files, transform=train_transforms)
        train_loader = DataLoader(train_ds, batch_size=1)

        test_ds = Dataset(data=test_files, transform=test_transforms)
        test_loader = DataLoader(test_ds, batch_size=1)
        print(train_loader)
        return train_loader, test_loader
        


prepare(in_dir='/Users/tylerklimas/Desktop/LiverSegmentation/LiverSegmentationData/groups/') 

56


(<monai.data.dataloader.DataLoader at 0x7fe15b3e6b20>,
 <monai.data.dataloader.DataLoader at 0x7fe15b3e6a00>)

In [19]:
train_loader, test_loader = prepare(in_dir='/Users/tylerklimas/Desktop/LiverSegmentation/LiverSegmentationData') 


In [24]:
data = list(train_loader)
data


[]